In [15]:
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random
import pickle

In [19]:
HTML('<script src="http://d3js.org/d3.v3.min.js" charset="utf-8"></script>')
HTML('<script src="http://labratrevenge.com/d3-tip/javascripts/d3.tip.v0.6.3.js"></script>')

In [20]:
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')

In [21]:
css_text = '''

.bar {
  fill: steelblue;
}

.bar:hover {
  fill: brown;
}

.axis {
  font: 10px sans-serif;
}

.axis path,
.axis line {
  fill: none;
  stroke: #000;
  shape-rendering: crispEdges;
}

.x.axis path {
  display: none;
}

.d3-tip {
  line-height: 1;
  font-weight: bold;
  padding: 12px;
  background: rgba(0, 0, 0, 0.8);
  color: #fff;
  border-radius: 2px;
}

'''

In [22]:
js_text_template = Template('''

var margin = {top: 20, right: 20, bottom: 30, left: 60},
    width = 650 - margin.left - margin.right,
    height = 300 - margin.top - margin.bottom;

var x = d3.scale.ordinal()
    .rangeRoundBands([0, width], .1);

var y = d3.scale.linear()
    .range([height, 0]);

var xAxis = d3.svg.axis()
    .scale(x)
    .orient("bottom");

var yAxis = d3.svg.axis()
    .scale(y)
    .orient("left");
    
var tip = d3.tip()
  .attr('class', 'd3-tip')
  .offset([-10, 0])
  .html(function(d) {
    return "<strong>Pjengs:</strong> <span style='color:red'>" + formatValue(d.y) + " kr." + "</span>";
  })

var svg = d3.select("#graph-div").append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

var formatValue = d3.format(",.0f");

svg.call(tip);

var data = $data ;

  x.domain(data.map(function(d) { return d.letter; }));
  y.domain([0, d3.max(data, function(d) { return d.y; })]);

  svg.append("g")
      .attr("class", "x axis")
      .attr("transform", "translate(0," + height + ")")
      .call(xAxis);

  svg.append("g")
      .attr("class", "y axis")
      .call(yAxis);

  svg.selectAll(".bar")
      .data(data)
    .enter().append("rect")
      .attr("class", "bar")
      .attr("x", function(d) { return x(d.letter); })
      .attr("width", x.rangeBand())
      .attr("y", function(d) { return y(d.y); })
      .attr("height", function(d) { return height - y(d.y); })
      .on('mouseover', tip.show)
      .on('mouseout', tip.hide);

function type(d) {
  d.y = +d.y;
  return d;
}
''')

In [23]:
js_text_template_2 = Template('''

var bars = svg.selectAll(".bar").data($data);
  
bars
  .transition()
    .attr("y", function(d) { return y(d.y); })
    .attr("height", function(d) { return height - y(d.y); });

''')

In [7]:
data = pd.DataFrame({'letter': ['Laun','Húsnæðisstuðningur','Barnabætur','Ráðstöfunartekjur'], 'y': [1,1,1,1]})
data.head()

,letter,y
0,Laun,1
1,Húsnæðisstuðningur,1
2,Barnabætur,1
3,Ráðstöfunartekjur,1


In [8]:
js_text = js_text_template.substitute({'data': json.dumps(data.to_dict(orient='records'))})
HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))

In [9]:
data['y'] = [random.uniform(0,1) for d in data['y']]
js_text = js_text_template_2.substitute({'data': json.dumps(data.to_dict(orient='records'))})
HTML('<script>' + js_text + '</script>')

In [16]:
# loading data
pickle_in = open("einstaedur_oryrki.pickle","rb")
example_dict = pickle.load(pickle_in)

In [24]:
data =\
    pd.DataFrame({'letter':['Laun e skatt',
                            'Fjármagnstekjur',
                            'Húsnæðisstuðningur',
                            'Barnabætur',
                            'Örorkubætur',
                            'Ráðstöfunartekjur'],
                  'y':[example_dict[0]['tekjur_e_skatt'],
                       example_dict[0]['fjarmagnstekjur_e_skatt'],
                       example_dict[0]['husnaedisstudningur'],
                       example_dict[0]['barnabaetur'],
                       example_dict[0]['ororka_e_skatt'],
                       example_dict[0]['pjeng_i_vasa']]})
data

,letter,y
0,Laun e skatt,260374.400000
1,Fjármagnstekjur,20000.000000
2,Húsnæðisstuðningur,32688.103090
3,Barnabætur,64535.425227
4,Örorkubætur,84137.737526
5,Ráðstöfunartekjur,460335.665842


In [25]:
js_text = js_text_template.substitute({'data': json.dumps(data.to_dict(orient='records'))})
HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))